# Tidying Billboard Data

In this cute module, I will clean up some data to make it "tidy" in the words of the accomplished Hadley Wickham. The work is based on his paper https://vita.had.co.nz/papers/tidy-data.pdf, which defines tidy data as "structuring datasets to facilitate analysis." It is important to note that although the tidy data may not be the best form to store in a database (we will often find duplications of some kind), it is perhaps the optimal way to massage it to feed it into our models.

We will use pandas to melt this dataset to get it in tidy form which consists of these three principles from Wickham's paper:

    - Each variable forms a column.
    - Each observation forms a row.
    - Each type of observational unit forms a table.
    
Before we take on this project, I'll tell you a little bit about the dataset. It has shape $(317,81)$, where each row represents a song that has features year, artist, track, name, date entered, and $76$ weeks that show the rank of the song if it has entered the top $100$. If the song drops off the top $100$ list, its rank will be substituted with a NaN. Relatively straight forward, but a perfect example for some tidying.

Now, let us embark.
***

In [1]:
import pandas as pd
import numpy as np

billboard = pd.read_csv('./raw_data/billboard.csv')

In [2]:
billboard.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***
## What's Wrong With This Picture?

We have a bunch of column headers that are $\textbf{values}$ rather than $\textbf{variables}$. In particular, these are the $wk$ variables, that range from the $1^{st}$ week to the $76^{th}$. How could we possibly run a model on this? Although running a model to try to predict weekly ratings seems infeasible to begin with (we would essentially have a time series on our hands, but each track would be its own time series with a small number of datapoints), it is made even more abstract with the current structure.

Hence, we would like to melt the data (turn it from short and wide to long and skinny) so that there is a $week$ column that takes on all $76$ values, and a $rating$ column that displays the corresponding rank of the track. The code is simple and follows here:
***

In [3]:
billboard_melt = billboard.melt(id_vars   = ['year', 'artist', 'track', 'time', 'date.entered'],
                               var_name   = 'week',
                               value_name = 'rating')

***
The $id\_vars$ parameter corresponds to the columns for which we would like to keep the same. These will be duplicated when we melt, as I will show below. Now, with these specified columns unchanged, intuitively, the only thing that can change is converting the remaining column names to variables in a new column, and each corresponding rating to be a value associated with each week. The $var\_name$ parameter assigns a name to the $week$ column (the new column that has the weeks as variables), and $value\_name$ assigns a name to each week's corresponding rating.
***

In [4]:
billboard_melt.head()

,year,artist,track,time,date.entered,week,rating
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0


***
We can see that this new structure adheres to Wickham's notion of tidy data: every variable is a column, and every observation is a row. In this case, the third criterion is not accounted for. For $\textit{modeling}$ purposes, this data is tidy, but for $\textit{database}$ purposes, this is not tidy, since we actually have multiple types of observational units being stored in one table. This melted dataset it is actually what we would like to have after doing a few joins of separate tables from a database, but this will be addressed below.

Let's verify the shape of this melted dataset. We originally had $317$ rows, and each of the $76$ weeks now constitutes its own row. Hence, we should have $317\textrm{ x }76 = 24092$ rows, and a total shape of $(24092, 7)$; indeed it is so:
***

In [5]:
billboard_melt.shape

(24092, 7)

***
Earlier, it was hinted that melting would cause duplications. In this case, we would have $76$ rows for each song. Here is an illustration of this phenomenon.
***

In [6]:
print('Number of rows with the track "Wobble Wobble":', billboard_melt[billboard_melt['track'] == 'Loser'].shape[0], '\n')
print(billboard_melt[billboard_melt['track'] == 'Wobble Wobble'].head())
print('...')
print(billboard_melt.loc[billboard_melt['track'] == 'Wobble Wobble'].iloc[71:])

Number of rows with the track "Wobble Wobble": 76 

      year    artist          track  time date.entered week  rating
4     2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk1    57.0
321   2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk2    34.0
638   2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk3    25.0
955   2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk4    17.0
1272  2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk5    17.0
...
       year    artist          track  time date.entered  week  rating
22511  2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk72     NaN
22828  2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk73     NaN
23145  2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk74     NaN
23462  2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk75     NaN
23779  2000  504 Boyz  Wobble Wobble  3:35   2000-04-15  wk76     NaN


***
## So What About Database Storage?

Now that our data is tidy, we are ready to run our models. I don't think there is enough here to actually run one, and quite honestly that is not the point of this quick module. So, let's take a different approach and dabble in database design. How $\textit{should}$ we organize this for a database? Is the melted dataset optimal?

I've hinted at this above, but no it's not. There are too many duplications of song metadata, which can become nasty (and pointless) when storing. So, we would like to decompose this data into a few tables such that the melted dataset would be the product of joining our decomposed tables together.

I think it makes sense that we should have two tables: one that has a track id (primary key) and all corresponding info about the track (year, artist, track, time, date entered), and another table that has track id as the linking foreign key to the week and rating. In essence, this is the process of $\textbf{normalizing}$ the data. The final shapes should be $(317,6)$ and $(24092,3)$, respectively.
***

In [7]:
billboard_songs       = billboard.iloc[:,:5]
billboard_songs['id'] = np.arange(len(billboard_songs))

In [8]:
billboard_songs.head()

,year,artist,track,time,date.entered,id
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,1
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,2
3,2000,3 Doors Down,Loser,4:24,2000-10-21,3
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,4


In [9]:
billboard_ratings = billboard_melt.merge(billboard_songs,
                                         on = ['year', 'artist', 'track', 'time', 'date.entered']                             
                                        )

In [10]:
billboard_ratings.head()

,year,artist,track,time,date.entered,week,rating,id
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0,0
1,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk2,82.0,0
2,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk3,72.0,0
3,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk4,77.0,0
4,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk5,87.0,0


In [11]:
# Now, we only want to keep the three columns, ridding ourselves of unnecessary duplication

billboard_ratings = billboard_ratings[['id', 'week', 'rating']]

In [12]:
billboard_ratings.head()

,id,week,rating
0,0,wk1,87.0
1,0,wk2,82.0
2,0,wk3,72.0
3,0,wk4,77.0
4,0,wk5,87.0


***
This is much better. Even though we still have a dataset with ~24000 rows, the duplications come as one column of foreign keys rather than five columns of repeated song metadata.

Just to beat a dead horse, lets look at the memory usage of each of our datasets. The full, melted, redundant dataset uses significantly more memory than the other two, as we would expect.
***

In [13]:
billboard_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24092 entries, 0 to 24091
Data columns (total 7 columns):
year            24092 non-null int64
artist          24092 non-null object
track           24092 non-null object
time            24092 non-null object
date.entered    24092 non-null object
week            24092 non-null object
rating          5307 non-null float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.3+ MB


In [14]:
billboard_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 6 columns):
year            317 non-null int64
artist          317 non-null object
track           317 non-null object
time            317 non-null object
date.entered    317 non-null object
id              317 non-null int64
dtypes: int64(2), object(4)
memory usage: 15.0+ KB


In [15]:
billboard_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24092 entries, 0 to 24091
Data columns (total 3 columns):
id        24092 non-null int64
week      24092 non-null object
rating    5307 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 752.9+ KB


***
Well, that about does it! Thank you for taking the time to go through this rather elementary implementation of data tidying! Even though it is simple, the concepts are sound, and the methodology is an important one.

Please let me know if you have other ideas/questions, and be sure to take a peak at the other notebooks in this repository that address other issues common in untidy data.
***